In [1]:
import numpy as np
import pandas as pd
import json
import gzip

from os import walk
from scipy.spatial import KDTree

In [2]:
LBPath = r'C:\Users\marvi\Desktop\SchemaOrgData\LocalBusiness_top100' + '\\'
LBFilenames = next(walk(LBPath), (None, None, []))[2]
LBData = []

for lb in LBFilenames[:20]:
    with gzip.open(LBPath + lb, 'r') as dataFile:
        for line in dataFile:
            lineData = json.loads(line.decode('utf-8'))
            LBData.append(lineData)
lbDF = pd.DataFrame(LBData)


In [3]:
# convert Geo into two columns (long + lat)
geoData = lbDF[lbDF['geo'].notnull()]
data = pd.concat([geoData, geoData['geo'].apply(pd.Series)], axis=1)

In [4]:
# Cut out entries that only have one geo-value
# Very few entrys only have either longitude or latitude
data = data[data['longitude'].notna()]
data = data[data['latitude'].notna()]

In [5]:
#pip install geopy
#replace , with . to solve Formation issues
lon = 'longitude'
lat = 'latitude'
data[lon] = data[lon].str.replace(',','.')
data[lat] = data[lat].str.replace(',','.')


In [26]:
def createKDTree(tupleArray):
    tree = KDTree(tupleArray)
    return tree

# Return all values that are in a specific proximity
def queryTree(tree, point, neighbours = 2):    
    point = [float(i) for i in point]
    #idx = tree.query_ball_point(point, r=proximity)
    idx = tree.query(point, k=neighbours)
    return idx[1]

# convert to tuples and from string to float
lonArr = data[lon].to_numpy()
latArr = data[lat].to_numpy()
tupleArray = np.array((lonArr, latArr)).T.astype('float32')

# create new column with all matching points
tree = createKDTree(tupleArray)
idx = queryTree(tree, tupleArray[0])

# Search for the closest neighbour in all of the points
data['MatchingPoints'] = data.apply(lambda row: queryTree(tree,[row[lon], row[lat]]), axis=1) 



In [ ]:
# install if necessary
# pip install geopy

In [68]:
import geopy.distance

def calcDifference(pointOne, pointTwo):
    return geopy.distance.great_circle(pointOne, pointTwo).km

In [69]:
data['Distance'] = data['MatchingPoints'].apply(lambda row: calcDifference( (data.iloc[row[0]][lat], data.iloc[row[0]][lon]),(data.iloc[row[1]][lat], data.iloc[row[1]][lon])))
data['Distance'] = pd.to_numeric(data['Distance'])

In [76]:
data['Distance'].dtypes

dtype('float64')

In [67]:
one = 0
two = 1

pointOne = (data.iloc[one][lat], data.iloc[one][lon])
pointTwo = (data.iloc[two][lat], data.iloc[two][lon])

calcDifference(pointOne, pointTwo)

# Result = Distance Bielefeld - Leipzig 

Distance(274.567370042535)

In [86]:
data[data['Distance'] < 0.005]

,row_id,name,address,telephone,page_url,aggregaterating,email,geo,url,image,...,3,4,5,6,7,8,latitude,longitude,MatchingPoints,Distance
21400,27,Apollo-Optik,"{'addressregion': 'Rheinland-Pfalz', 'addresscountry': 'DE', 'addresslocality': 'Mainz', 'postalcode': '55116', 'streetaddress': 'Stadthausstr. 1'}",(06131)&nbsp;693876,https://www.11880.com/branchenbuch/mainz-am-rhein/110633957B26871248/apollo-optik.html,"{'reviewcount': '12', 'bestrating': '5', 'worstrating': '1', 'ratingvalue': '3.33'}",service@apollo-optik.com,"{'latitude': '50,001', 'longitude': '8,270224'}",https://www.apollo.de/store-7053/mainz,https://cdn.11880.com/apollo-optik_17032642_mw240h70_mainz-am-rhein.jpg,...,NaN,NaN,NaN,NaN,NaN,NaN,50.001,8.270224,"[12, 10272]",0.000000
21451,78,iDT Ihr-Dienstleistungs-Team Region Dortmund,"{'streetaddress': 'Ruhrallee 9', 'postalcode': '44139', 'addresscountry': 'DE', 'addresslocality': 'Dortmund', 'addressregion': 'Nordrhein-Westfalen'}",None,https://www.11880.com/branchenbuch/dortmund/060371176B101600855/idt-ihr-dienstleistungs-team-region-dortmund.html,None,thimma@idt-online.de,"{'latitude': '51,508139', 'longitude': '7,467584'}",https://www.idt-online.de,//static.11880.com/Portal/images/11880/photo-upload.png,...,NaN,NaN,NaN,NaN,NaN,NaN,51.508139,7.467584,"[47, 1640]",0.000000
21464,91,Ralf und Doris Aschauer Montageservice + Innenausbau,"{'addressregion': 'Baden-Württemberg', 'addresscountry': 'DE', 'streetaddress': 'Uhlandstr. 10', 'postalcode': '72666', 'addresslocality': 'Neckartailfingen'}",(07127)&nbsp;237871,https://www.11880.com/branchenbuch/neckartailfingen/062393753B43330557/ralf-und-doris-aschauer-montageservice-innenausbau.html,None,None,"{'longitude': '9,270464', 'latitude': '48,611063'}",https://www.dvag.de,//static.11880.com/Portal/images/11880/photo-upload.png,...,NaN,NaN,NaN,NaN,NaN,NaN,48.611063,9.270464,"[58, 8703]",0.000000
21484,111,Dr.med. Oliver M. Mainusch Facharzt für Dermatologie,"{'addresslocality': 'Velbert', 'addresscountry': 'DE', 'streetaddress': 'Blumenstr. 1', 'addressregion': 'Nordrhein-Westfalen', 'postalcode': '42551'}",(02051)&nbsp;57783,https://www.11880.com/branchenbuch/velbert/110070223B41514886/dr-med-oliver-m-mainusch-facharzt-fuer-dermatologie.html,"{'reviewcount': '2', 'bestrating': '5', 'worstrating': '1', 'ratingvalue': '1.00'}",None,"{'longitude': '7,04722', 'latitude': '51,338768'}",http://www.hautconnection.de,//static.11880.com/Portal/images/11880/photo-upload.png,...,NaN,NaN,NaN,NaN,NaN,NaN,51.338768,7.04722,"[76, 2399]",0.000000
21511,138,Anwaltskanzlei Linden und Mosel Rechtsanwälte,"{'addresscountry': 'DE', 'addresslocality': 'Köln', 'streetaddress': 'Zülpicher Str. 274', 'postalcode': '50937', 'addressregion': 'Nordrhein-Westfalen'}",(0221)&nbsp;422220,https://www.11880.com/branchenbuch/koeln/081314253B29398591/anwaltskanzlei-linden-und-mosel-rechtsanwaelte.html,"{'ratingvalue': '5.00', 'bestrating': '5', 'worstrating': '1', 'reviewcount': '1'}",info@lindenundmosel.de,"{'longitude': '6,923274', 'latitude': '50,922934'}",https://lindenundmosel.de,https://cdn.11880.com/anwaltskanzlei-linden-und-mosel-rechtsanwaelte_23487112_mw240h70_koeln.jpg,...,NaN,NaN,NaN,NaN,NaN,NaN,50.922934,6.923274,"[99, 120]",0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203042,7048,"Baja . Gospodarstwo agroturystyczne, sklep","{'addresslocality': 'Swornegacie', 'streetaddress': 'Mestwina 6', 'addressregion': 'woj. pomorskie', 'postalcode': '89-608'}",52 398 11 71,https://www.biznesfinder.pl/pomorskie%2C%20s%C5%82upski%2C%20retowo/gospodarstwo%20agroturystyczne,None,baja55@poczta.onet.pl,"{'latitude': '53.8596725490196', 'longitude': '17.4955469411765'}",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,53.8596725490196,17.4955469411765,"[89946, 86574]",0.003986
203063,7069,Almax. ZUHP. Kędzior J.,"{'postalcode': '39-300', 'streetaddress': 'Wojska Polskiego 3', 'addresslocality': 'Mielec', 'addressregion': 'woj. podkarpackie'}",17 788 61 16,https://www.

In [87]:
print(data.iloc[89995])
print(data.iloc[83446])

row_id                       7097                                                                                                                               
name                         Eleryk. Krzysztof Eleryk                                                                                                           
address                      {'postalcode': '05-250', 'addresslocality': 'Marki', 'streetaddress': 'Al. Piłsudskiego 123B', 'addressregion': 'woj. mazowieckie'}
telephone                    501 712 705                                                                                                                        
page_url                     https://www.biznesfinder.pl/marki-eleryk-krzysztof-eleryk-5269555.html                                                             
aggregaterating              None                                                                                                                               
email                        konta